# Comparing Pandas, Polars, DASK and DuckDB

In [1]:
import time
import pandas as pd
import polars as pl
import dask.dataframe as dd
import duckdb

In [2]:
csv_file = "data/dataset.csv"
parquet_file = "data/dataset.parquet"

### Pandas

In [2]:
def bare_pandas():
    # Timing Pandas
    start_time = time.time()
    pandas_df = pd.read_csv(csv_file)
    top_pandas = pandas_df.groupby("client_id")["reading"].mean().nlargest(10)
    pandas_end_time = time.time() - start_time
    print(f"Pandas Time: {pandas_end_time} seconds")
    print(f"Top Pandas Consumers:\n{top_pandas}\n")


# bare_pandas()

### Pola.rs

In [6]:
def bare_polars():
    start_time = time.time()
    polars_df = pl.read_csv(csv_file)
    top_polars = (
        polars_df.group_by("client_id")
        .agg(pl.mean("reading"))
        .sort("reading", descending=True)
        .head(10)
    )
    polars_end_time = time.time() - start_time
    print(f"Polars Time: {polars_end_time} seconds")
    print(f"Top Polars Consumers:\n{top_polars}\n")


# bare_polars()

: 

### Streamed Pola.rs

In [16]:
start_time = time.time()
df = (
    pl.scan_csv(csv_file)
    .select("client_id", "reading")
    .group_by("client_id")
    .agg(pl.mean("reading"))
    .sort("reading", descending=True)
    .collect()
)
polars_scan_end_time = time.time() - start_time
df

client_id,reading
i64,f64
4495,513.933888
8164,513.760061
3796,513.531397
4778,513.426086
616,513.04239
7945,512.390231
3472,512.097417
13002,512.095028
3027,511.962784


### DASK

In [17]:
start_time = time.time()
dask_df = dd.read_csv(csv_file)
top_dask = dask_df.groupby("client_id")["reading"].mean().nlargest(10).compute()
dask_end_time = time.time() - start_time
print(f"Dask Time: {dask_end_time} seconds")
top_dask

Dask Time: 11.801136016845703 seconds


client_id
4495     513.933888
8164     513.760061
3796     513.531397
4778     513.426086
616      513.042390
7945     512.390231
3472     512.097417
13002    512.095028
3027     511.962784
10926    511.949987
Name: reading, dtype: float64

In [8]:
# Timing DuckDB
duckdb.sql(
    """
    SELECT
        client_id,
        avg(reading) as mean,
        count(*) as count
    FROM
        read_csv_auto('data/dataset.csv')
    GROUP BY
        client_id
    ORDER BY
        mean DESC
    LIMIT 10;
    """
)

┌───────────┬────────────────────┬───────┐
│ client_id │        mean        │ count │
│   int64   │       double       │ int64 │
├───────────┼────────────────────┼───────┤
│      4495 │  513.9338879107169 │  6775 │
│      8164 │  513.7600611476672 │  6624 │
│      3796 │  513.5313974840019 │  6742 │
│      4778 │  513.4260864237832 │  6918 │
│       616 │  513.0423898277211 │  6689 │
│      7945 │   512.390230810469 │  6674 │
│      3472 │  512.0974170476485 │  6661 │
│     13002 │  512.0950282726445 │  6680 │
│      3027 │ 511.96278439717895 │  6703 │
│     10926 │ 511.94998678644055 │  6581 │
├───────────┴────────────────────┴───────┤
│ 10 rows                      3 columns │
└────────────────────────────────────────┘

# What if we replace CSV with parquet?

### Bare Pandas on parquet

In [19]:
def bare_pandas_on_parquet():
    # Timing Pandas
    start_time = time.time()
    pandas_df = pd.read_parquet(parquet_file)
    top_pandas = pandas_df.groupby("client_id")["reading"].mean().nlargest(10)
    pandas_end_time = time.time() - start_time
    print(f"Pandas Time: {pandas_end_time} seconds")
    print(f"Top Pandas Consumers:\n{top_pandas}\n")
    top_pandas


# bare_pandas_on_parquet()

: 

## Bare Polars on Parquet

In [ ]:
def bare_polars_on_parq():
    start_time = time.time()
    polars_df = pl.read_parquet(parquet_file)
    top_polars = (
        polars_df.group_by("client_id")
        .agg(pl.mean("reading"))
        .sort("reading", descending=True)
        .head(10)
    )
    polars_end_time = time.time() - start_time
    print(f"Polars Time: {polars_end_time} seconds")
    print(f"Top Polars Consumers:\n{top_polars}\n")


# bare_polars_on_parq()

: 

## Streamed Polars on Parquet

In [5]:
start_time = time.time()
df = (
    pl.scan_parquet(parquet_file)
    .select("client_id", "reading")
    .group_by("client_id")
    .agg(pl.mean("reading"))
    .sort("reading", descending=True)
    .collect()
)
polars_scan_parquet_end_time = time.time() - start_time
df.head(10)

client_id,reading
i64,f64
4495,513.933888
8164,513.760061
3796,513.531397
4778,513.426086
616,513.04239
7945,512.390231
3472,512.097417
13002,512.095028
3027,511.962784


### DASK on Parquet

In [8]:
start_time = time.time()
dask_df = dd.read_parquet(parquet_file)
top_dask = dask_df.groupby("client_id")["reading"].mean().nlargest(10).compute()
dask_parquet_end_time = time.time() - start_time
print(f"Dask Time: {dask_parquet_end_time} seconds")
top_dask

Dask Time: 4.806739091873169 seconds


client_id
4495     513.933888
8164     513.760061
3796     513.531397
4778     513.426086
616      513.042390
7945     512.390231
3472     512.097417
13002    512.095028
3027     511.962784
10926    511.949987
Name: reading, dtype: float64

: 

### DuckDB on Parquet

In [4]:
duckdb.sql(
    """
    SELECT
        client_id,
        avg(reading) as mean,
        count(*) as count
    FROM
        read_parquet('data/dataset.parquet')
    GROUP BY
        client_id
    ORDER BY
        mean DESC
    LIMIT 10;
    """
)

┌───────────┬────────────────────┬───────┐
│ client_id │        mean        │ count │
│   int64   │       double       │ int64 │
├───────────┼────────────────────┼───────┤
│      4495 │  513.9338879107169 │  6775 │
│      8164 │  513.7600611476671 │  6624 │
│      3796 │  513.5313974840019 │  6742 │
│      4778 │  513.4260864237833 │  6918 │
│       616 │   513.042389827721 │  6689 │
│      7945 │  512.3902308104689 │  6674 │
│      3472 │  512.0974170476486 │  6661 │
│     13002 │  512.0950282726446 │  6680 │
│      3027 │ 511.96278439717884 │  6703 │
│     10926 │  511.9499867864405 │  6581 │
├───────────┴────────────────────┴───────┤
│ 10 rows                      3 columns │
└────────────────────────────────────────┘